In [ ]:
#from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff

In [ ]:
#Resizing images, if needed
SIZE_X = 128 
SIZE_Y = 128
n_classes=4 #Number of classes for segmentation
num_images = 200  #Total 1600 available

In [ ]:
# image_names = glob.glob("E:/STUDY_MATERIALS/L4-T2/CSE 472/Unet/sandstone_data_for_ML/full_labels_for_deep_learning/128_patches/*.tif")
# print(len(image_names))
# image_names.sort()
# image_names_subset = image_names[0:num_images]
# images = [cv2.imread(image, 0) for image in image_names_subset]
# read tiff images
images = tiff.imread("./PhC-C2DH-U373/01/*.tif")
# sort images by name
images.sort()
# select first 200 images
images = images[0:num_images]
image_dataset = np.array(images)
image_dataset = np.expand_dims(image_dataset, axis = 3)
print(image_dataset.shape)

In [ ]:
mask = tiff.imread("./sandstone_data_for_ML/full_labels_for_deep_learning/128_patches/masks_as_128x128_patches.tif")
mask.sort()
mask_subset = mask[0:num_images]

mask_dataset = np.array(mask_subset)


In [ ]:
print("Image data shape is: ", image_dataset.shape)
print("Mask data shape is: ", mask_dataset.shape)
print("Max pixel value in image is: ", image_dataset.max())
print("Labels in the mask are : ", np.unique(mask_dataset))

In [ ]:
# data augmentation
import elasticdeform
sigma = 30  # the standard deviation of the Gaussian filter
alpha = 50  # the intensity of the deformation
deformed_image = elasticdeform.deform_grid(image_dataset, sigma=sigma, alpha=alpha)


In [ ]:
#Encode labels to 0, 1, 2, 3, ... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = mask_dataset.shape  
mask_dataset_reshaped = mask_dataset.reshape(-1,1)
mask_dataset_reshaped_encoded = labelencoder.fit_transform(mask_dataset_reshaped)
mask_dataset_encoded = mask_dataset_reshaped_encoded.reshape(n, h, w)

np.unique(mask_dataset_encoded)

In [ ]:
mask_dataset_encoded = np.expand_dims(mask_dataset_encoded, axis = 3)
print(mask_dataset_encoded.shape)

In [ ]:
#Normalize images
image_dataset = image_dataset /255.  #Can also normalize or scale using MinMax scaler

In [ ]:
#Split training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset_encoded, test_size = 0.2, random_state = 42)

In [ ]:
from keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))

test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))



In [ ]:
# Building Unet by dividing encoder and decoder into blocks

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate


def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)   #Not in the original network. 
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)  #Not in the original network
    x = Activation("relu")(x)

    return x

#Encoder block: Conv block followed by maxpooling


def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p   

#Decoder block
#skip features gets input from encoder for concatenation

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

#Build Unet using the blocks
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024) #Bridge

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:  #Binary
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d4)  #Change the activation based on n_classes
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
# Import some libraries
import os
import shutil
from PIL import Image
import numpy as np

# Configure data path for celltracking data set PhC-C2DH-U373 and file structure
path_dataset = "./PhC-C2DH-U373/"
path_filestructure = "./data"

# Initialize file structure
if not os.path.exists(path_filestructure): os.mkdir(path_filestructure)

# Iterate over both data sets
for ds in ["01", "02"]:
    # Define image directories
    path_ds_img = os.path.join(path_dataset, ds)
    path_ds_seg = os.path.join(path_dataset, ds + "_GT", "SEG")
    # Obtain sample list
    sample_list = os.listdir(path_ds_seg)
    # Remove every file which does not match image typ and preprocess sample names
    for i in reversed(range(0, len(sample_list))):
        if not sample_list[i].endswith(".tif"):
            del sample_list[i]
        else:
            sample_list[i] = sample_list[i][7:]
    # Iterate over each sample and transform the data into desired file structure
    for sample in sample_list:
        index = ds + "_" + sample[:-4]
        # Create sample directory
        path_sampleDir = os.path.join(path_filestructure, index)
        if not os.path.exists(path_sampleDir): os.mkdir(path_sampleDir)
        # Copy image file into filestructure
        path_ds_sample_img = os.path.join(path_ds_img, "t" + sample)
        path_fs_sample_img = os.path.join(path_sampleDir, "imaging.tif")
        shutil.copy(path_ds_sample_img, path_fs_sample_img)
        # Copy segmentation file into filestructure
        seg_file = "man_seg" + sample
        path_ds_sample_seg = os.path.join(path_ds_seg, seg_file)
        path_fs_sample_seg = os.path.join(path_sampleDir, "segmentation.tif")
        # Load segmentation from file
        seg_raw = Image.open(path_ds_sample_seg)
        # Convert segmentation from Pillow image to numpy matrix
        seg_pil = seg_raw.convert("LA")
        seg = np.array(seg_pil)
        # Keep only intensity and remove maximum intensitiy range
        seg_data = seg[:,:,0]
        # Union all separate cell classes to a single one
        seg_data[seg_data > 0] = 1
        # Transform numpy array back to a Pillow image & save to disk
        seg = Image.fromarray(seg_data)
        seg.save(path_fs_sample_seg, format="TIFF")